In [305]:
import pandas as pd
import re
# The user has requested to parse an xlsx file to get specific data.
# As there are multiple files uploaded, we will first list the files to ensure we are parsing the correct one.
# The user has mentioned "All Cycles" page, so we will look for that sheet in the Excel files.

# Let's define the path to the uploaded xlsx files.
file_paths = [
    r"C:\Users\s1814897\Projects\BMGPlateReaderPython\example_inputs\3_11_2023.xlsx",
    r"C:\Users\s1814897\Projects\BMGPlateReaderPython\example_inputs\27_10_2023.xlsx",
    r"C:\Users\s1814897\Projects\BMGPlateReaderPython\example_inputs\31_10_2023-raw-data.xlsx",
]
# The user has provided new information that the relevant data starts at row 10.

def parse_excel_with_header(file, header_row):
    try:
        # Read the 'All Cycles' sheet with header starting from row 10 (index 9 as it's 0-indexed)
        data = pd.read_excel(file, sheet_name='All Cycles', header=header_row - 1)
        data.columns = [col.replace('\n', ' ') for col in data.columns]  # Replace newlines in column names
        
        # Check if the required columns are in the dataframe
        misc_columns = ['Content']

        # Overwrite the first row for each column in required_columns
        for column in misc_columns:
            if column in data.columns:
                data.at[0, column] = column  # Replace the value in the first row with the column name
        
        # Identify 'Raw Data' columns
        raw_data_columns = [col for col in data.columns if col.startswith("Raw Data")]
        selected_columns = misc_columns + raw_data_columns
        
        # Filter out any columns that aren't in the dataframe
        selected_columns = [col for col in selected_columns if col in data.columns]

        # Select only the columns that match the criteria
        parsed_data = data[selected_columns]

        # Set the first row as the header
        parsed_data.columns = parsed_data.iloc[0]
        parsed_data = parsed_data.drop(parsed_data.index[0])
        parsed_data.columns = parsed_data.columns.str.strip()

        # # Rename and clean up the Content column
        parsed_data.rename(columns={'Content': 'Sample'}, inplace=True)
        parsed_data['Sample'] = parsed_data['Sample'].str.replace('Sample ', '')

        # Convert columns to numeric, handling non-numeric values
        for col in parsed_data.columns[1:]:  # Exclude the 'Sample' column
            parsed_data[col] = pd.to_numeric(parsed_data[col], errors='coerce')
        # Parse time strings in column names to minutes
        parsed_data.columns = [time_to_minutes(col) if "Sample" not in col else col for col in parsed_data.columns]

        return parsed_data
    
    except Exception as e:
        print(f"Error processing file {file}: {e}")
    return None


def time_to_minutes(time_str):
    """Converts time strings to minutes."""
    h_match = re.search(r'(\d+)\s*h', time_str)
    m_match = re.search(r'(\d+)\s*min', time_str)

    hours = int(h_match.group(1)) if h_match else 0
    minutes = int(m_match.group(1)) if m_match else 0

    return hours * 60 + minutes

def parse_excel_with_header(file, header_row):
    try:
        # Read the 'All Cycles' sheet with header starting from row 10
        data = pd.read_excel(file, sheet_name='All Cycles', header=header_row - 1)
        data.columns = [col.replace('\n', ' ') for col in data.columns]

        # Overwrite the first row for each column in misc_columns
        misc_columns = ['Content']
        for column in misc_columns:
            if column in data.columns:
                data.at[0, column] = column

        # Identify 'Raw Data' columns
        raw_data_columns = [col for col in data.columns if col.startswith("Raw Data")]
        selected_columns = misc_columns + raw_data_columns

        # Filter out any columns that aren't in the dataframe
        selected_columns = [col for col in selected_columns if col in data.columns]

        # Select only the columns that match the criteria
        parsed_data = data[selected_columns]

        # Set the first row as the header and clean up
        parsed_data.columns = parsed_data.iloc[0]
        parsed_data = parsed_data.drop(parsed_data.index[0])
        parsed_data.columns = parsed_data.columns.str.strip()
        parsed_data.rename(columns={'Content': 'Sample'}, inplace=True)
        parsed_data['Sample'] = parsed_data['Sample'].str.replace('Sample ', '')

        # Convert columns to numeric, handling non-numeric values
        for col in parsed_data.columns[1:]:
            parsed_data[col] = pd.to_numeric(parsed_data[col], errors='coerce')
        # Parse time strings in column names to minutes
        parsed_data.columns = [time_to_minutes(col) if "Sample" not in col else col for col in parsed_data.columns]

        # Transpose the dataframe
        parsed_data = parsed_data.set_index('Sample').T
        return parsed_data

    except Exception as e:
        print(f"Error processing file {file}: {e}")
    return None

def subtract_blanks(data, groups, blanks):
    # Calculate the average for each blank and check for empty data frames
    average_blanks = {}
    for b in blanks:
        blank_data = data[data['Sample'] == b]
        if not blank_data.empty:
            average_blanks[b] = blank_data.mean(numeric_only=True)
    

    # Subtract the average blank values from the appropriate group samples
    for group in groups:
        for sample in group:
            if sample in blanks:
                continue  # Skip blanks
            blank_sample = next((b for b in blanks if b in group), None)
            if blank_sample and blank_sample in average_blanks:
                for col in data.columns[1:]:  # Exclude the 'Sample' column
                    data.loc[data['Sample'] == sample, col] -= average_blanks[blank_sample][col]
                    
    # Drop rows that are in blanks
    data = data[~data['Sample'].isin(blanks)]

    return data

def subtract_blanks(transposed_data, groups, blanks):
    # Calculate the average for each blank and check for empty data frames
    average_blanks = {}
    for b in blanks:
        if b in transposed_data.columns:
            blank_data = transposed_data[b]
            average_blanks[b] = blank_data.mean()

    # Subtract the average blank values from the appropriate group samples
    for group in groups:
        for sample in group:
            if sample in blanks:
                continue  # Skip blanks
            blank_sample = next((b for b in blanks if b in group), None)
            if blank_sample and blank_sample in average_blanks:
                transposed_data[sample] -= average_blanks[blank_sample]

    # Drop columns that are in blanks
    transposed_data.drop(columns=blanks, inplace=True, errors='ignore')

    return transposed_data

In [ ]:
# Transpose the 

In [303]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def plot_with_error_bars(data, samples, error_type='std'):
    """
    Plots line plots for each group with error bars at each timepoint.

    :param data: DataFrame where samples are rows.
    :param groups: List of lists, each sublist contains sample names belonging to a group.
    :param error_type: 'std' for standard deviation, 'sem' for standard error of the mean.
    """
    plt.figure(figsize=(10, 6))

    for sample in samples:

        # Select only the samples in the current group
        sample_data = data[data['Sample'] == sample].iloc[:, 1:].apply(pd.to_numeric, errors='coerce')


        # Creating the DataFrame
        restructured_data = {
            "Time (minutes)": sample_data.columns,
            "Replicate 1": sample_data.iloc[0],
            "Replicate 2": sample_data.iloc[1],
            "Replicate 3": sample_data.iloc[2],
        }

        df = pd.DataFrame(restructured_data)
        for col in df.columns:  # Exclude the 'Sample' column
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Calculating mean and standard deviation
        df_mean = df.mean(axis=1)
        df_std = df.std(axis=1)
        # Plotting the line plot with area filled with standard deviation
        # errors = df.std() if error_type == 'std' else df.sem()

        # # Plotting the line plot with error bars
        # plt.errorbar(x=df["Time (minutes)"], y=df_mean, yerr=errors, label=f'Group {groups.index(group) + 1}', fmt='-o')

        plt.plot(df["Time (minutes)"], df_mean, label='Mean Value')
        plt.fill_between(df["Time (minutes)"], df_mean-df_std, df_mean+df_std, alpha=0.3)

        # # Calculate mean and error for each time point in the sample
        # mean_values = sample_data.mean()
        # error = sample_data.std() if error_type == 'std' else sample_data.sem()

        # # Plotting with error bars
        # plt.errorbar(x=sample_data.columns, y=mean_values, yerr=error, label=sample, fmt='-o')

    plt.xlabel('Time')
    plt.ylabel('Measurement')
    plt.title('Group Measurements with Error Bars')
    plt.legend()
    plt.show()

def plot_with_error_bars(data, samples, error_type='std'):
    """
    Plots line plots for each sample with error bars at each timepoint.

    :param data: DataFrame where samples are rows and time points are columns.
    :param samples: List of sample names.
    :param error_type: 'std' for standard deviation, 'sem' for standard error of the mean.
    """
    plt.figure(figsize=(10, 6))

    for sample in samples:
        # Select only the data for the current sample
        sample_data = data.loc[sample].apply(pd.to_numeric, errors='coerce')

        # Calculate mean and standard deviation (or SEM)
        mean_values = sample_data.mean(axis=0)
        errors = sample_data.std(axis=0) if error_type == 'std' else sample_data.sem(axis=0)

        # Plotting the line plot with error bars
        plt.errorbar(x=sample_data.columns, y=mean_values, yerr=errors, label=sample, fmt='-o')

    plt.xlabel('Time (Minutes)')
    plt.ylabel('Measurement')
    plt.title('Sample Measurements with Error Bars')
    plt.legend()
    plt.grid(True)
    plt.show()

In [322]:

data = parse_excel_with_header(file, 10)
data.index.name = "Minutes"
data
# NEED TOW WRITE blanks subraction for transposed table
# then plotting


Sample,X1,X1,X1,X2,X2,X2,X3,X3,X3,X4,...,X8,X8,X9,X9,X9,X10,X10,X10,X11,X11
Minutes,,,,,,,,,,,,,,,,,,,,,
0,641,643,636,640,650,637,627,634,611,490,...,501,503,524,530,518,323,319,325,280,260
10,576,572,589,576,567,561,557,563,547,434,...,448,447,459,456,453,489,491,483,278,263
20,565,568,575,545,539,532,546,545,524,532,...,427,428,422,417,418,1473,1498,1466,270,263
30,557,565,568,530,525,525,523,539,527,663,...,413,419,410,411,402,2931,2960,2914,275,259
40,562,570,576,519,524,523,527,538,528,744,...,413,417,394,397,392,4352,4432,4322,268,261
50,566,565,573,527,517,520,533,534,528,795,...,407,413,404,392,385,5649,5773,5612,271,260
60,572,572,581,529,524,525,537,545,533,817,...,416,414,400,401,387,6817,6937,6802,274,261
70,578,578,595,534,540,524,552,553,538,819,...,407,410,400,398,396,7874,8062,7797,274,264
80,577,579,585,542,542,533,556,564,543,814,...,418,414,403,399,400,8822,9035,8792,275,258


In [304]:
# Attempt to parse the Excel files with the new header information
file = file_paths[-1]
print(file)
# for file in file_paths:
data = parse_excel_with_header(file, 10)
groups = [
    ['X1','X2','X3'],
    ['X4','X5','X6'],
    ['X7','X8','X9'],
    ['X10','X11'],
]
blanks = ['X3','X6','X9','X11']


data = subtract_blanks(data, groups, blanks)
# display(data)
# data.to_csv("temp.csv", index=False)
# Example usage
samples = [
'X1','X2',
'X4','X5',
'X7','X8',
'X10',
]

plot_with_error_bars(data,samples, error_type='std')


C:\Users\s1814897\Projects\BMGPlateReaderPython\example_inputs\31_10_2023-raw-data.xlsx


KeyError: 'X1'

<Figure size 1000x600 with 0 Axes>

In [302]:
data

,Sample,0,10,20,30,40,50,60,70,80,90,100,110,120
1,X1,17.000000,20.333333,26.666667,27.333333,31.000000,34.333333,33.666667,30.333333,22.666667,23.0,32.666667,24.000000,26.000000
2,X1,19.000000,16.333333,29.666667,35.333333,39.000000,33.333333,33.666667,30.333333,24.666667,18.0,33.666667,22.000000,24.000000
3,X1,12.000000,33.333333,36.666667,38.333333,45.000000,41.333333,42.666667,47.333333,30.666667,29.0,37.666667,29.000000,33.000000
4,X2,16.000000,20.333333,6.666667,0.333333,-12.000000,-4.666667,-9.333333,-13.666667,-12.333333,-19.0,-14.333333,-16.000000,-24.000000
5,X2,26.000000,11.333333,0.666667,-4.666667,-7.000000,-14.666667,-14.333333,-7.666667,-12.333333,-15.0,-10.333333,-8.000000,-17.000000
6,X2,13.000000,5.333333,-6.333333,-4.666667,-8.000000,-11.666667,-13.333333,-23.666667,-21.333333,-28.0,-17.333333,-19.000000,-19.000000
10,X4,19.333333,42.666667,159.333333,301.333333,387.333333,431.000000,454.333333,453.666667,449.666667,456.0,459.333333,453.000000,460.000000
11,X4,19.333333,48.666667,164.333333,303.333333,380.333333,421.000000,443.333333,452.666667,460.666667,465.0,455.333333,455.000000,446.000000
12,X4,19.333333,38.666667,162.333333,304.333333,380.333333,424.000000,434.333333,452.666667,463.666667,468.0,458.333333,463.000000,462.000000
13,X5,17.333333,28.666667,89.333333,135.333333,184.333333,206.000000,226.333333,238.666667,245.666667,262.0,264.333333,269.000000,276.000000


In [289]:
# Creating the DataFrame
data = {
    "Time (minutes)": [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120],
    "Replicate 1": [17.0, 20.333333, 26.666667, 27.333333, 31.0, 34.333333, 33.666667, 30.333333, 22.666667, 23.0, 32.666667, 24.0, 26.0],
    "Replicate 2": [19.0, 16.333333, 29.666667, 35.333333, 39.0, 33.333333, 33.666667, 30.333333, 24.666667, 18.0, 33.666667, 22.0, 24.0],
    "Replicate 3": [12.0, 33.333333, 36.666667, 38.333333, 45.0, 41.333333, 42.666667, 47.333333, 30.666667, 29.0, 37.666667, 29.0, 33.0]
}

pd.DataFrame(data)

,Time (minutes),Replicate 1,Replicate 2,Replicate 3
0,0,17.000000,19.000000,12.000000
1,10,20.333333,16.333333,33.333333
2,20,26.666667,29.666667,36.666667
3,30,27.333333,35.333333,38.333333
4,40,31.000000,39.000000,45.000000
5,50,34.333333,33.333333,41.333333
6,60,33.666667,33.666667,42.666667
7,70,30.333333,30.333333,47.333333
8,80,22.666667,24.666667,30.666667
9,90,23.000000,18.000000,29.000000


In [218]:
data

,Sample,0,10,20,30,40,50,60,70,80,90,100,110,120
1,X1,17.000000,20.333333,26.666667,27.333333,31.000000,34.333333,33.666667,30.333333,22.666667,23.0,32.666667,24.000000,26.000000
2,X1,19.000000,16.333333,29.666667,35.333333,39.000000,33.333333,33.666667,30.333333,24.666667,18.0,33.666667,22.000000,24.000000
3,X1,12.000000,33.333333,36.666667,38.333333,45.000000,41.333333,42.666667,47.333333,30.666667,29.0,37.666667,29.000000,33.000000
4,X2,16.000000,20.333333,6.666667,0.333333,-12.000000,-4.666667,-9.333333,-13.666667,-12.333333,-19.0,-14.333333,-16.000000,-24.000000
5,X2,26.000000,11.333333,0.666667,-4.666667,-7.000000,-14.666667,-14.333333,-7.666667,-12.333333,-15.0,-10.333333,-8.000000,-17.000000
6,X2,13.000000,5.333333,-6.333333,-4.666667,-8.000000,-11.666667,-13.333333,-23.666667,-21.333333,-28.0,-17.333333,-19.000000,-19.000000
10,X4,19.333333,42.666667,159.333333,301.333333,387.333333,431.000000,454.333333,453.666667,449.666667,456.0,459.333333,453.000000,460.000000
11,X4,19.333333,48.666667,164.333333,303.333333,380.333333,421.000000,443.333333,452.666667,460.666667,465.0,455.333333,455.000000,446.000000
12,X4,19.333333,38.666667,162.333333,304.333333,380.333333,424.000000,434.333333,452.666667,463.666667,468.0,458.333333,463.000000,462.000000
13,X5,17.333333,28.666667,89.333333,135.333333,184.333333,206.000000,226.333333,238.666667,245.666667,262.0,264.333333,269.000000,276.000000


In [152]:

X1X1X1X2X2X2